In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from tqdm import tqdm

In [2]:
class LocalizationNetwork(nn.Module):
    def __init__(self, input_channels):
        super(LocalizationNetwork, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=7)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, kernel_size=5)

        # Placeholder fc1 (will update after shape calculation)
        self.fc1 = nn.Linear(1, 32)  # Temporary placeholder
        self.fc2 = nn.Linear(32, 6)  # 6 affine parameters

        # Initialize weights for identity transformation
        self.fc2.weight.data.zero_()
        self.fc2.bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Dynamically compute the flattened size
        if not hasattr(self, 'computed_fc1'):
            flattened_size = x.view(x.shape[0], -1).shape[1]
            self.fc1 = nn.Linear(flattened_size, 32).to(x.device)
            self.computed_fc1 = True  # Prevent re-initialization

        x = x.view(x.shape[0], -1)  # Flatten
        x = F.relu(self.fc1(x))
        theta = self.fc2(x)
        theta = theta.view(-1, 2, 3)
        return theta


class WeightingModule(nn.Module):
    def __init__(self, sigma=0.5):
        super(WeightingModule, self).__init__()
        self.lambda_param = nn.Parameter(torch.tensor(0.5))
        self.sigma = sigma

    def forward(self, grid):
        dist2 = grid[..., 0] ** 2 + grid[..., 1] ** 2
        weight = 1 + self.lambda_param * torch.exp(-dist2 / (2 * self.sigma ** 2))
        return weight.unsqueeze(-1)


class DEFTModule(nn.Module):
    def __init__(self, input_channels, sigma=0.5):
        super(DEFTModule, self).__init__()
        self.localization = LocalizationNetwork(input_channels)
        self.weighting = WeightingModule(sigma)

    def forward(self, x):
        theta = self.localization(x)
        grid = F.affine_grid(theta, x.size(), align_corners=False)
        weight = self.weighting(grid)
        x_transformed = F.grid_sample(x, grid, align_corners=False)

        if x.shape[1] > 1:
            weight = weight.expand(-1, x.shape[2], x.shape[3], x.shape[1]).permute(0, 3, 1, 2)
        else:
            weight = weight.permute(0, 3, 1, 2)

        x_weighted = x_transformed * weight
        return x_weighted

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_model = resnet50(weights=True)
resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])  # Remove classification layer
resnet_model.eval().to(device)

# Load DEFT Module
deft_model = DEFTModule(input_channels=3).to(device)  # Input channels = 3 (RGB)
deft_model.eval()
###########################################################
# Define Image Transformation
###########################################################

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

C:\Users\PAWANESH\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Paths
rgb_path = "D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04"
flow_path = "D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04"
label_csv = "D:/Datasets/Datasets/EPIC_Kitchen/Label/P01_04.csv"

# Load CSV
df = pd.read_csv(label_csv)

In [8]:
# Feature extraction function
def extract_features(img_path):
    try:
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            image = deft(image)
            feature = model(image).squeeze().cpu().numpy()
        return feature
    except Exception as e:
        print(f"Skipping {img_path}: {e}")
        return None
# Sample every 5th frame
S = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Process frames
features = []
all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Processing RGB + Flow frames"):
    rgb_frame_path = os.path.join(rgb_path, frame)
    flow_frame_path = os.path.join(flow_path, frame)  # flow has same name

    rgb_feat = extract_features(rgb_frame_path)
    flow_feat = extract_features(flow_frame_path)

    if rgb_feat is not None and flow_feat is not None:
        frame_number = int(frame.split("_")[-1].split(".")[0])

        label_row = df[(df['StartFrame'] <= frame_number) & (df['EndFrame'] >= frame_number)]

        if not label_row.empty:
            action_label = label_row.iloc[0]['Action_class']
            combined = [frame, action_label] + rgb_feat.tolist() + flow_feat.tolist()
            features.append(combined)

# Save to CSV
if features:
    num_rgb = len(rgb_feat)
    num_flow = len(flow_feat)
    columns = ['Frame', 'Action_class'] + \
              [f'RGB_Feature_{i}' for i in range(num_rgb)] + \
              [f'Flow_Feature_{i}' for i in range(num_flow)]

    out_df = pd.DataFrame(features, columns=columns)
    os.makedirs("../../Features", exist_ok=True)
    out_df.to_csv("../../Features/Feature_P01_04_RGB+Flow.csv", index=False)
    print("✅ RGB + Flow features saved to Feature_P01_04_RGB+Flow.csv")
else:
    print("⚠️ No features extracted.")

Processing RGB + Flow frames:   1%|▍                                                   | 5/631 [00:00<00:14, 43.33it/s]

Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_00000.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00000.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_00000.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_00010.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00010.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_00010.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_00020.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00020.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_00020.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_00030.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00030.jpg:

Processing RGB + Flow frames:   2%|█▏                                                 | 15/631 [00:00<00:15, 40.65it/s]

Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00080.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_00080.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_00090.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00090.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_00090.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_00100.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00100.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_00100.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_00110.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_00110.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_0

Processing RGB + Flow frames:   4%|██                                                 | 25/631 [00:00<00:14, 40.55it/s]

...

Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06150.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06150.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06150.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06160.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06160.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06160.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06170.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06170.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06170.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06180.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06180.jpg:

Processing RGB + Flow frames:  99%|█████████████████████████████████████████████████▋| 627/631 [00:18<00:00, 29.69it/s]

Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06220.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06220.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06220.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06230.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06230.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06230.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06240.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06240.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06240.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06250.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06250.jpg:

Processing RGB + Flow frames: 100%|██████████████████████████████████████████████████| 631/631 [00:18<00:00, 33.32it/s]

Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06280.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06280.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06280.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06290.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06290.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06290.jpg'
Skipping D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04\frame_06300.jpg: name 'deft' is not defined
Skipping D:/Datasets/Datasets/EPIC_Kitchen/Flow/P01_04\frame_06300.jpg: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\Flow\\P01_04\\frame_06300.jpg'
⚠️ No features extracted.
